## Merge shloka, vyakyanam and translation of Shrikaanta Charitam

Given two types of word files, 
- one with shloka and it vyakyanam  - **text image './01-sams-merged-raw.txt'**
- the other with shloka and it's traslation with footnotes - **text image './02-eng-trans-fn-raw.txt'**
- this program generates a single file with shloka, vyakyanam ,translation, footnotes and traslitered shloka - **text image './sc-compiled.tsv'**
- from the manually generate text images of the doc files ( using export , cut/paste etc) and midly annotating tge text files

In [2]:

# read a file and return a list of lines
import re
import pandas as pd
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

def dn_to_en(x):
    return transliterate(x, sanscript.DEVANAGARI, sanscript.IAST)

def read_file(file_name):
    lines = []
    with open(file_name, 'r') as f:
        for line in f:
            lines.append(re.sub(r"\|\|\s*$" , r"॥", line))
            # lines.append(line)
    return ''.join(lines)

str_ = read_file('./01-sams-merged-raw.txt')  
str_ = re.sub(r"(.*)॥\s*([१२३४५६७८९०]+)\s*॥(.*?)$", r"\1{{\2}}\3॥", str_, flags=re.MULTILINE)  # ॥१॥ to {{1}}॥ sloka numberin
str_ = re.sub(r"(.*)॥\s*(\d+)\s*॥(.*?)$", r"\1{{\2}}\3॥", str_, flags=re.MULTILINE) # ॥8888॥ to {{8888}}॥ manual numbering of sarga start (8888) ,sarga end (9999) , magic for pre sarga1,shloka1(०१)
str_ = re.sub("॥", "॥\n", str_, flags=re.MULTILINE)
lines = [ x for x in str_.split('\n') if x.strip() != '']
df = pd.DataFrame(lines, columns=['text']).assign(
    sloka_num = lambda x: x.text.apply(lambda y: re.search('{{\s*(\S+)\s*}}', y).group(1).strip() if bool(re.search('{{\s*\S+\s*}}', y)) else False).apply ( lambda y: int(dn_to_en(y)) if y else -1)
    , chap_seq = lambda x: x.sloka_num.apply(lambda y: y==8888).cumsum()
)
pd.set_option('display.max_colwidth', 200)
df[df.sloka_num.apply( lambda x: x in [1111, 8888, 1,2,151,152, 9999]) & df.chap_seq.apply(lambda x: x in [1,2,24,25])].sort_index().style.set_caption('<h3>Sloka Numbering <br><h4> few samples </h4>')


,text,sloka_num,chap_seq
33,प्रथमः सर्गः {{8888}} ॥,8888,1
43,सावधाना भवन्त्वत्र मम स्खलितयोजने {{०१}}॥,1,1
46,यस्यान्तिके शुभ्रदशानिवेशश्रियं किरीटेन्दुकराः श्रयन्ते {{१}} ॥,1,1
49,धूमानभिज्ञोऽपि रतेरजस्रमश्रुस्रुतेर्योऽजनि सूत्रधारः {{२}} ॥,2,1
237,नमस्कारवर्णनं नाम प्रथमः सर्गः {{9999}} ॥,9999,1
239,द्वितीयः सर्गः {{8888}}॥,8888,2
241,ये क्षीरनीरप्रविभागदक्षा विवेकिनस्ते कवयो जयन्ति {{१}} ॥,1,2
244,सच्चक्रमव्याजविराजमानतैक्ष्ण्यप्रकर्षं यदि नाम न स्यात् {{२}} ॥,2,2
479,सुजनदुर्जनवर्णनो नाम द्वितीयः सर्गः {{9999}} ॥,9999,2
5501,चतुर्विंशः सर्गः {{8888}} ॥,8888,24


In [24]:
sloka_index = df[df.sloka_num>0].index
sloka_num = df[df.sloka_num>0].sloka_num
sloka_index_next = pd.Series(sloka_index).shift(-1).fillna(-3).astype(int)
sloka_index_prev = pd.Series(sloka_index).shift(1).fillna(-3).astype(int)
acc=[]
nx = 0
for curr, next, prev, snum  in zip(sloka_index, sloka_index_next, sloka_index_prev, sloka_num): 
    if next<0: continue
    if prev<0: continue
    slice = df.iloc[prev+1:curr+2].copy()
    sloka_num = slice.sloka_num.max()
    chap_num = slice.chap_seq.max()

    sloka = ''.join(slice.iloc[1:-1].text.to_list())
    if sloka_num == 1 and chap_num>1 :
        sloka = acc[-1][-1] + sloka
    vyakhya = slice.iloc[-1].text
    if nx>=0:
        acc.append([nx, chap_num, sloka_num, prev+1,curr+2, sloka, '', vyakhya])
        # if (nx> 0.99*len(sloka_index)) or (nx< 0.01*len(sloka_index)): display([nx, prev+1, curr+2], slice)
    # if nx>1500 : break
    nx+=1

compiled = pd.DataFrame(acc, columns=['nx', 'chapter number', 'sloka number','prev', 'curr', 'sloka', 'transliterated', 'vyakhya']).assign(
    sloka = lambda x: x.sloka.apply(
                lambda y: re.sub(r"\{\{\s*", r"॥ ", y, flags=re.MULTILINE)
            ).apply(
                lambda y: re.sub(r"\}\}\s*", r" ", y, flags=re.MULTILINE)
            # ).apply(
                # lambda y: y.strip()
            ).apply(
                lambda y: re.sub(r"।", "।\n", y)
            ).apply(
                lambda y: re.sub(r"\s+", " ", y)
            )
).drop ( columns = ['nx', 'prev', 'curr']).iloc[1:].assign( 
    transliterated = lambda x: x.sloka.apply(lambda y: transliterate(y, sanscript.DEVANAGARI, sanscript.IAST))
)

compiled.vyakhya = compiled.apply(lambda x: '' if x['sloka number']>=8888 else x.vyakhya, axis=1)
# compiled.to_csv('./sc-compiled.tsv', sep='\t', index=False)
compiled[compiled['sloka number']<1000].head(5).style.set_caption(f'<h2> Compiled - श्लोक, Eng-transliterate, व्याख्या</h2>') #.set_properties(subset=['sloka'], **{'width': '1200px', 'align': 'center'})


,chapter number,sloka number,sloka,transliterated,vyakhya
1,1,1,जीयात्कृतानङ्गपतङ्गदाहः खट्वाङ्गिनो नेत्रशिखिप्रदीपः । यस्यान्तिके शुभ्रदशानिवेशश्रियं किरीटेन्दुकराः श्रयन्ते ॥ १ ॥,jīyātkṛtānaṅgapataṅgadāhaḥ khaṭvāṅgino netraśikhipradīpaḥ | yasyāntike śubhradaśāniveśaśriyaṃ kirīṭendukarāḥ śrayante || 1 ||,किरीटेन्दोर्मौलिचन्द्रस्य करा रश्मयो यस्य नेत्रशिखिनो निकटे शुभ्रदशाया वर्णा- न्तरसन्निधानाच्छ्वेततराया अवस्थाया वर्तेश्च निवेशस्तच्छोभां श्रयन्ते सेवन्ते सः । अनङ्गः काम एव पतङ्गः कीटविशेषस्तस्य दाहः स कृतो येन सः । खट्वाङ्गिनो महादेवस्य नेत्राग्निरेव प्रदीपो जीयात्सर्वोत्कृष्टो भूयात् । अग्नेरत्र काव्ये वर्णनीयत्रिपुरदाहसाधकत्वाज्जयप्राप्त्यर्थं तद्वारा जगतामपि जयप्राप्त्या सतामुत्साहनार्थं जीयादित्याशीःप्रयोगः । नेत्रशिखीति विशेषेण ग्रहणं भगवतस्त्रिपुरदाहेऽन्यानपेक्षत्वद्योतनार्थम् । तस्य भगवदादृतत्वेन । सर्वासामग्निशक्तीनां तदुत्थत्वात् । कृतानङ्गपतङ्गदाह इति विशेषणं व्यभिचाराभावेऽपि सङ्गतम् । जीयादित्यत्र हेतुरूपतयातस्योपन्यासात्। 'वक्षःस्थली रक्षतु सा जगन्ति जगत्प्रसूतेर्गरुडध्वजस्य ।' इत्यत्र जगत्प्रसूतेरितिवत् । अत्र नेत्रशिखिनो जीयादित्याशीर्वादेनानङ्गदाहः कारणम् । उपकृतो ह्युपकारिणमाशास्ते । उपकारः पुरदाहलक्षणोऽनुक्तोऽप्यापतितः । यो हि मनोभवं पतङ्गवद्दहति तस्य पुरदाहो दण्डापूपिकया सिद्धः। कामदाहात्पूर्वकालावस्थाया वा व्यवच्छेदार्थं विशेषणम् । कृततीर्थयात्रो मैत्रो वन्द्य इतिवत् । नह्यकृतकामदाहस्याग्नेर्जयाशिषा प्रकृतपोषकत्वम् । वर्णनीयस्य त्रिपुरदाहस्य तथाविधेन नेत्राग्निना कथञ्चिदप्यसूचितत्वात् । अथवा समरसोत्सुकस्य कवेः शिखी जीयादित्युक्तौ कृतानङ्गपतङ्गदाह इति विशेषणं हेतुः । तथाहि 'सर्वैः कैश्चन' (१ । ५७) इत्यादिनोत्तरग्रन्थेन स्वस्य शान्तत्वं प्रकाशिष्यते । अनङ्ग इति भविष्यदवस्थानुगुणस्य नाम्नः प्रयोगोऽनुप्रासानुरोधात्कृतः । कामस्य सुदहत्वावबोधार्थमुपन्यस्तस्य पतङ्गरूपस्यानुगुण्यानुरोधान्नेत्राग्नेर्दीपरूपणम् । खट्वाङ्गिन इत्ययोगव्यवच्छेदः । न त्वन्ययोगव्यवच्छेदः । खट्वाङ्गिन इति संहारानुगुणं नाम ग्रन्थारम्भेऽपि निर्दिष्टम् । कामदाहानुगुणत्वात् । शुभ्रदशासहितश्च दीप: पतङ्गं दहति ॥
2,1,2,तनोतु भूतिं दहतादघानि स शूलिनो लोचनपावको वः । धूमानभिज्ञोऽपि रतेरजस्रमश्रुस्रुतेर्योऽजनि सूत्रधारः ॥ २ ॥,tanotu bhūtiṃ dahatādaghāni sa śūlino locanapāvako vaḥ | dhūmānabhijño'pi raterajasramaśrusruteryo'jani sūtradhāraḥ || 2 ||,लोकोत्तरप्रभावत्वात्समिनिरपेक्षमेव ज्वलन्नत एव धूमस्यानभिज्ञोऽपि यो रतेः कामवध्वा बाष्पप्रवाहस्य सूचनात्सूत्रधारः समपादि स हरस्य नेत्राग्निभूतिं विस्तारयतु । पापानि दहतात् । भूतिरैश्वर्यं भस्म च । पावकानुगुणत्वाद्दहतादित्याद्युक्त्युपन्यासः । शूलिनः पावको भूतिं तनोतीति वैचित्र्यप्रतीत्यर्थः शूलिशब्दोपन्यासः । अत्र बाष्पहेतौ धूमेऽसत्यपि बाष्पस्य फलस्योत्पत्त्या विभावनालङ्कारः ॥
3,1,3,भालस्थलीरङ्गतले मृडस्य हुताशनस्ताण्डवकृत्स वोऽव्यात् । यस्मिन्रतिप्राणसमः शरीरमुन्मालकायेव निजं मुमोच ॥ ३ ॥,bhālasthalīraṅgatale mṛḍasya hutāśanastāṇḍavakṛtsa vo'vyāt | yasminratiprāṇasamaḥ śarīramunmālakāyeva nijaṃ mumoca || 3 ||,रतेः प्राणसमः प्रियः कामो नृत्तप्रीतः सन्नुन्मालकाय पारितोषकदानायेव शरीरं यस्मिंस्तत्त्याज स पावको हरस्य ललाटभूमिरेव रङ्गतलं तत्र ताण्डवमिव ताण्डवं कर्ता युष्मान्प्रीणयतु । रतिप्राणसम इति पर्यायवक्रतया कामुकत्वावगमेनोन्मालकौदार्यप्रतीतिः ॥
4,1,4,तन्नेत्रमासूत्रितवह्निसख्यमाख्यातु वः शर्म शिवस्य यस्मिन् । ज्वालावलीपल्लवकेलितल्पे विनैव रत्या मदनो निदद्रौ ॥ ४ ॥,tannetramāsūtritavahnisakhyamākhyātu vaḥ śarma śivasya yasmin | jvālāvalīpallavakelitalpe vinaiva ratyā madano nidadrau || 4 ||,यस्मिन्नेत्रे ज्वालावल्येव वर्णसन्निवेशाभ्यां पल्लवलीलाशयनं तत्र रतिरहितः कामो निदद्रौ दग्धः। तदासूत्रितमुट्टङ्कितं वह्निना सख्यं येन तच्छिवसम्बन्धि नेत्रं वः सुखं प्रथयताम् । मदन इति पर्यायवक्रतया कामुकत्वावगतौ सत्यां विनैव रत्येत्येवकारोपन्याससङ्गतिः। तदुपोद्बलकं च ज्वालावलीनां पल्लवशय्यारूपणम् ॥
5,1,5,भर्गस्य सीमन्तितभालभित्तिः स्वाहापतिर्वो दुरितं दुनोतु । यत्तापयुक्त्येव विलीयमानो न पूर्णतामेति किरीटचन्द्रः ॥ ५ ॥,bhargasya sīmantitabhālabhittiḥ svāhāpatirvo duritaṃ dunotu | yattāpayuktyeva vilīyamāno na pūrṇatāmeti kirīṭacandraḥ || 5 ||,यत्तापयोगेन विलीयमानो विगलन्मुकुटचन्द्रः पूर्णो न भवति सः । भर्गस्य । सीमन्तिता मध्यस्थाने निवेशेन द्विधा कृता भालभित्तिर्येन सः । स्वाहापतिरग्निर्वः पापं दहतु । अत्र मौलीन्दुपूर्णत्व

In [33]:
canto = -1
trans = []
fns = []
for x in [ x.strip() for x in read_file('./02-eng-trans-fn-raw.txt').split("\n") if x.strip() != '' and x.strip()[0] != '#'] :
    canto_pattern = re.search(r"^Canto(\d+)", x)
    if canto_pattern:
        canto = int(canto_pattern.groups()[0])
        # continue

    trans_pattern = re.search(r"^(\d+).*?(\w.*?)\[(\-?\d+)\]", x + ' [-1]')
    if trans_pattern:
        (sloka_num, translation, fn_num) = [int(trans_pattern.groups()[0]), trans_pattern.groups()[1], int(trans_pattern.groups()[2])]
        trans.append([canto, sloka_num, fn_num, translation, x[:10] + ' ... ' + x[-10:]])
        # continue 

    fns_pattern = re.search(r"^\[(\d+)\].*?(\w.*)", x)
    if fns_pattern:
        (fn_num, fn_txt) = [int(fns_pattern.groups()[0]), fns_pattern.groups()[1]]
        fns.append([canto, fn_num, fn_txt, x[:10] + ' ... ' + x[-10:]])
        # continue

trans_df = pd.DataFrame (trans, columns=['chapter number', 'sloka number', 'fn number', 'translation' ,'_raw_translation']).sort_values(['chapter number', 'sloka number', 'fn number']).reset_index(drop=True)
fns_df = pd.DataFrame (fns, columns=['chapter number', 'fn number', 'fn text', '_raw_footnote']).sort_values(['chapter number', 'fn number']).reset_index(drop=True)
trans_fn = pd.merge( trans_df, fns_df, on=['chapter number', 'fn number'], how='left')
trans_fn.head(10).style.set_caption(f'<h2> Translations and Footnotes </h2>')

,chapter number,sloka number,fn number,translation,_raw_translation,fn text,_raw_footnote
0,1,1,1,"May the fire of Shiva’s (third) eye, standing for a lamp that burnt to ashes the locust in the form of Kama, reign supreme. In its vicinity, the rays of the moon on Shiva’s crest appear lovely like an arrangement of white wicks.",1. May the ... wicks.[1],"The poem begins with a praise of Shiva’s ocular fire to highlight the importance of fire in the destruction of the Tripuras, to indicate that all fires, including the fire of Shiva’s arrow, owe their origin to his ocular fire, and to suggest that the fire which destroyed even Kama as if he were a mere locust can easily destroy the Tripuras. The use of the word ‘anaṅga’ for Kama, that describes his state after being burnt down by Shiva, is for the purpose of alliteration (with ‘pataṅga’ and ‘khaṭvāṅgino’) (JR).",[1]The poe ... no’) (JR).
1,1,2,2,"May the fire of Shiva’s (third) eye grant you prosperity/sacred ash and burn down your sins – a fire that, in spite of being smokeless, stage-managed Rati’s incessant shedding of tears.",2. May the ... tears.[2],"The fire of Shiva’s third eye blazes forth smokeless in the absence of a feeding fuel because of its supernatural power. The figure of speech employed in this verse is ‘vibhāvanā’ since the effect - the production of tears – materializes even in the absence of the cause - the appearance of smoke. The use of the verb form ‘dahatāt’ (‘may it burn down’) is appropriate for the subject, namely, fire (JR).",[2]The fir ... fire (JR).
2,1,3,3,"May the Fire, dancing a robust dance on the stage that is Shiva’s forehead, save us. Kama, Rati’s sweetheart, offered his very body to it (i.e., the Fire-dancer) as a gift, so it seemed.",3. May the ... seemed.[3],"Unmālaka, a rare word, is explained by JR as ‘pāritoṣakapradāna’, ‘offering a gift’. The use of ‘ratiprāṇasama’ (‘as dear as life to Rati’) to describe Kama underscores the latter’s amorous nature and thereby a predilection for offering gifts (JR).",[3] Unmāla ... ifts (JR).
3,1,4,4,"Let Shiva’s eye – one that has forged a friendship with fire – proclaim happiness to you. In the pleasure bed of fresh leaves formed by its (i.e., the eye’s) billowing flames, Kama fell into a deep sleep, even without Rati by his side.",4. Let Sh ... s side.[4],The flames are similar to fresh leaves in shape and colour alike (JR).,[4] The fl ... like (JR).
4,1,5,-1,"May the fire that passes through the centre of Shiva’s rampart-like forehead and divides it into two halves destroy your sins. The moon on the Lord’s crest, melting constantly as it were by coming into contact with its heat, never attains fullness.",5. May the ... fullness.,nan,nan
5,1,6,5,"Let the fire of Shiva’s arrow reduce your sin to ashes. Like a second Aurva fire, it never had enough of the flood of tears shed by long-eyed Danava damsels.",6. Let the ... amsels.[5],"Aurva is the submarine fire ‘whose fuel is water’ (jalendhana). The use of ‘tṛpyati’ with the genitive ‘bāṣpāmbupūrasya’ is in accordance to the rule ‘pūraṇaguṇa---‘(Aṣṭādhyāyī, 2.2.11) (JR). Just as no amount of water is enough for the submarine fire, so no amount of teary water shed by the (widowed) wives of Danavas can satisfy the fire of Shiva’s arrow. The overall purport of the verse is that Shiva’s fiery arrow is never tired of exterminating the Danavas.",[5]Aurva i ... e Danavas.
6,1,7,6,"May Shiva, the universal teacher, save us. The Celestial River, by way of her row of revolving eddies, seems to circumambulate around him even though he (himself) bears her (respectfully) upon his head.",7. May Shi ... s head.[6],"The adjective ‘śiraḥsthā’ is used in the dual sense of ‘situated on the head’ and ‘respected’. By describing River Ganga as circumambulating Shiva in spite of the latter honouring her by placing her on his head, the poet conveys the special respect she has for him (JR).",[6]The adj ... him (JR).
7,1,8,7,"May Shiva’s boisterous laughs – bright as ambrosia – d

In [35]:
compiled_trans_fn = pd.merge( compiled, trans_fn, on=['chapter number', 'sloka number'], how='left').drop( columns=['fn number', '_raw_translation', '_raw_footnote']).fillna("N/A").reset_index(drop=True)
compiled_trans_fn.index = range(1, len(compiled_trans_fn) + 1) # type: ignore
compiled_trans_fn.to_csv('./sc-compiled.tsv', sep='\t', index=False)
compiled_trans_fn.head(10).style.set_caption(f'<h2> Compiled - श्लोक, Eng-transliterate, व्याख्या, Translations, Footnotes </h2>')

,chapter number,sloka number,sloka,transliterated,vyakhya,translation,fn text
1,1,1,जीयात्कृतानङ्गपतङ्गदाहः खट्वाङ्गिनो नेत्रशिखिप्रदीपः । यस्यान्तिके शुभ्रदशानिवेशश्रियं किरीटेन्दुकराः श्रयन्ते ॥ १ ॥,jīyātkṛtānaṅgapataṅgadāhaḥ khaṭvāṅgino netraśikhipradīpaḥ | yasyāntike śubhradaśāniveśaśriyaṃ kirīṭendukarāḥ śrayante || 1 ||,किरीटेन्दोर्मौलिचन्द्रस्य करा रश्मयो यस्य नेत्रशिखिनो निकटे शुभ्रदशाया वर्णा- न्तरसन्निधानाच्छ्वेततराया अवस्थाया वर्तेश्च निवेशस्तच्छोभां श्रयन्ते सेवन्ते सः । अनङ्गः काम एव पतङ्गः कीटविशेषस्तस्य दाहः स कृतो येन सः । खट्वाङ्गिनो महादेवस्य नेत्राग्निरेव प्रदीपो जीयात्सर्वोत्कृष्टो भूयात् । अग्नेरत्र काव्ये वर्णनीयत्रिपुरदाहसाधकत्वाज्जयप्राप्त्यर्थं तद्वारा जगतामपि जयप्राप्त्या सतामुत्साहनार्थं जीयादित्याशीःप्रयोगः । नेत्रशिखीति विशेषेण ग्रहणं भगवतस्त्रिपुरदाहेऽन्यानपेक्षत्वद्योतनार्थम् । तस्य भगवदादृतत्वेन । सर्वासामग्निशक्तीनां तदुत्थत्वात् । कृतानङ्गपतङ्गदाह इति विशेषणं व्यभिचाराभावेऽपि सङ्गतम् । जीयादित्यत्र हेतुरूपतयातस्योपन्यासात्। 'वक्षःस्थली रक्षतु सा जगन्ति जगत्प्रसूतेर्गरुडध्वजस्य ।' इत्यत्र जगत्प्रसूतेरितिवत् । अत्र नेत्रशिखिनो जीयादित्याशीर्वादेनानङ्गदाहः कारणम् । उपकृतो ह्युपकारिणमाशास्ते । उपकारः पुरदाहलक्षणोऽनुक्तोऽप्यापतितः । यो हि मनोभवं पतङ्गवद्दहति तस्य पुरदाहो दण्डापूपिकया सिद्धः। कामदाहात्पूर्वकालावस्थाया वा व्यवच्छेदार्थं विशेषणम् । कृततीर्थयात्रो मैत्रो वन्द्य इतिवत् । नह्यकृतकामदाहस्याग्नेर्जयाशिषा प्रकृतपोषकत्वम् । वर्णनीयस्य त्रिपुरदाहस्य तथाविधेन नेत्राग्निना कथञ्चिदप्यसूचितत्वात् । अथवा समरसोत्सुकस्य कवेः शिखी जीयादित्युक्तौ कृतानङ्गपतङ्गदाह इति विशेषणं हेतुः । तथाहि 'सर्वैः कैश्चन' (१ । ५७) इत्यादिनोत्तरग्रन्थेन स्वस्य शान्तत्वं प्रकाशिष्यते । अनङ्ग इति भविष्यदवस्थानुगुणस्य नाम्नः प्रयोगोऽनुप्रासानुरोधात्कृतः । कामस्य सुदहत्वावबोधार्थमुपन्यस्तस्य पतङ्गरूपस्यानुगुण्यानुरोधान्नेत्राग्नेर्दीपरूपणम् । खट्वाङ्गिन इत्ययोगव्यवच्छेदः । न त्वन्ययोगव्यवच्छेदः । खट्वाङ्गिन इति संहारानुगुणं नाम ग्रन्थारम्भेऽपि निर्दिष्टम् । कामदाहानुगुणत्वात् । शुभ्रदशासहितश्च दीप: पतङ्गं दहति ॥,"May the fire of Shiva’s (third) eye, standing for a lamp that burnt to ashes the locust in the form of Kama, reign supreme. In its vicinity, the rays of the moon on Shiva’s crest appear lovely like an arrangement of white wicks.","The poem begins with a praise of Shiva’s ocular fire to highlight the importance of fire in the destruction of the Tripuras, to indicate that all fires, including the fire of Shiva’s arrow, owe their origin to his ocular fire, and to suggest that the fire which destroyed even Kama as if he were a mere locust can easily destroy the Tripuras. The use of the word ‘anaṅga’ for Kama, that describes his state after being burnt down by Shiva, is for the purpose of alliteration (with ‘pataṅga’ and ‘khaṭvāṅgino’) (JR)."
2,1,2,तनोतु भूतिं दहतादघानि स शूलिनो लोचनपावको वः । धूमानभिज्ञोऽपि रतेरजस्रमश्रुस्रुतेर्योऽजनि सूत्रधारः ॥ २ ॥,tanotu bhūtiṃ dahatādaghāni sa śūlino locanapāvako vaḥ | dhūmānabhijño'pi raterajasramaśrusruteryo'jani sūtradhāraḥ || 2 ||,लोकोत्तरप्रभावत्वात्समिनिरपेक्षमेव ज्वलन्नत एव धूमस्यानभिज्ञोऽपि यो रतेः कामवध्वा बाष्पप्रवाहस्य सूचनात्सूत्रधारः समपादि स हरस्य नेत्राग्निभूतिं विस्तारयतु । पापानि दहतात् । भूतिरैश्वर्यं भस्म च । पावकानुगुणत्वाद्दहतादित्याद्युक्त्युपन्यासः । शूलिनः पावको भूतिं तनोतीति वैचित्र्यप्रतीत्यर्थः शूलिशब्दोपन्यासः । अत्र बाष्पहेतौ धूमेऽसत्यपि बाष्पस्य फलस्योत्पत्त्या विभावनालङ्कारः ॥,"May the fire of Shiva’s (third) eye grant you prosperity/sacred ash and burn down your sins – a fire that, in spite of being smokeless, stage-managed Rati’s incessant shedding of tears.","The fire of Shiva’s third eye blazes forth smokeless in the absence of a feeding fuel because of its supernatural power. The figure of speech employed in this verse is ‘vibhāvanā’ since the effect - the production of tears – materializes even in the absence of the cause - the appearance of smoke. The use of the verb form ‘dahatāt’ (‘may it burn down’) is appropriate for the subject, namely, fire (JR)."
3,1,3,भालस्थलीरङ्गतले मृडस्य हुताशनस्ताण्डवकृत्स वोऽव्यात् । यस्मिन्रतिप्राणसमः शरीरमुन्मालका

In [145]:
# import re
# import pandas as pd

# from indic_transliteration import sanscript
# from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

# def doint(x,e):
#     try:
#         return int(x)
#     except:
#         print(e, x)
#         return -99

# # scan a file for given multi-line regex patterns
# # and return a list of lines that match the pattern
# def scan_file_for_pattern(file_name, patterns):
#     lines = []
#     with open(file_name, 'r') as f:
#         for line in f:
#             for pattern in patterns:
#                 if re.search(pattern, line):
#                     lines.append(line)
#     return lines

# #lines = scan_file_for_pattern('./01-sams-merged-raw.txt', ['॥(.){0,7}॥', '।'])
# lines = scan_file_for_pattern('./01-sams-merged-raw.txt', ['.*'])
# df = pd.DataFrame(lines, columns=['text']).assign(
#     line_number = lambda x: x.index,
#     line_length = lambda x: x.text.apply(len),
#     _sloka = lambda x: x.text.apply(lambda y: bool(re.search('॥\s*(.){0,7}\s*॥', y))),
#     this_sloka = lambda x: x.apply(lambda y: re.search('॥\s*((.){0,7}\s*)॥', y.text).group(1).strip() if y._sloka else None, axis=1),
#     next_sloka = lambda x: x.this_sloka.shift(-1),
#     sloka = lambda x: x.apply(lambda y: y.this_sloka if y._sloka else y.next_sloka, axis=1),
#     sarga = lambda x: x.next_sloka.apply(lambda y: 1 if y=='१' else 0).cumsum(),
#     text = lambda x: x.text.apply(lambda y: re.sub('\n', '', y).strip()),
#     _snum = lambda x: x.sloka.apply(lambda y: ''.join([ str(ord('1') + ord(e) - ord('१') - 48) for e in str(y)]) if y else '-1'),
#     snum = lambda x: x.apply(lambda y: doint(y._snum, y), axis=1),
#     # transliteration = lambda x: x.text.apply(lambda y: transliterate(y, sanscript.DEVANAGARI, sanscript.IAST)),
#     # sloka = lambda x: x.text.apply(lambda y: re.search('॥(.){0,7}॥', y).group(0)),
#     # sarga = lambda x: x.text.apply(lambda y: re.search('॥(.){0,7}॥', y).group(0)),
#     )

# # display df with text wrapping
# pd.set_option('display.max_colwidth', None)
# df[df.sloka.notnull()].head(20)
# df[df.sarga==17]
# df.iloc[3062-10:3062+10]
# df[df.sloka.notnull()].sarga.value_counts().sort_index()
# df[df.sloka.notnull()].reset_index(drop=True).to_csv('./02-sams-merged-cooked.csv', index=False)
# df[df.sloka.notnull()].reset_index(drop=True)#.to_csv('./02-sams-merged-cooked.csv', index=False)
# # df.dtypes
# acc=[]
# for (sarga, snum), _df in  df[df.sloka.notnull()].groupby(['sarga', 'snum']) :
#     acc.append([sarga, snum , ''.join(_df.apply(lambda x: x.text, axis=1).to_list())])

# pd.DataFrame(acc, columns=['sarga', 'snum', 'text']). assign ( 
#     transliterated = lambda x: x.text.apply(lambda y: transliterate(y, sanscript.DEVANAGARI, sanscript.IAST)),
# ).tail(50)
# df[df.sloka.notnull()]#.reset_index(drop=True).to_cs

In [15]:
sc2 = pd.read_csv(
    './sc-second-cut.csv', 
    # sep='\t', 
    on_bad_lines='warn',
    skip_blank_lines=True)
sc2

,#,chapter_number,sloka_number,sloka,transliterated,vyakhya,translation,fn_text
0,NaN,NaN,NaN,प्रथमः सर्गः ॥,NaN,NaN,First Canto,NaN
1,1.0,1.0,1,जीयात्कृतानङ्गपतङ्गदाहः खट्वाङ्गिनो नेत्रशिखिप्रदीपः । यस्यान्तिके शुभ्रदशानिवेशश्रियं किरीटेन्दुकराः श्रयन्ते ॥ १ ॥,jīyātkṛtānaṅgapataṅgadāhaḥ khaṭvāṅgino netraśikhipradīpaḥ | yasyāntike śubhradaśāniveśaśriyaṃ kirīṭendukarāḥ śrayante || 1 ||,किरीटेन्दोर्मौलिचन्द्रस्य करा रश्मयो यस्य नेत्रशिखिनो निकटे शुभ्रदशाया वर्णान्तरसन्निधानाच्छ्वेततराया अवस्थाया वर्तेश्च निवेशस्तच्छोभां श्रयन्ते सेवन्ते सः । अनङ्गः काम एव पतङ्गः कीटविशेषस्तस्य दा...,"May the fire of Shiva’s (third) eye, standing for a lamp that burnt to ashes the locust in the form of Kama, reign supreme. In its vicinity, the rays of the moon on Shiva’s crest appear lovely lik...","The poem begins with a praise of Shiva’s ocular fire to highlight the importance of fire in the destruction of the Tripuras, to indicate that all fires, including the fire of Shiva’s arrow, owe th..."
2,2.0,1.0,2,तनोतु भूतिं दहतादघानि स शूलिनो लोचनपावको वः । धूमानभिज्ञोऽपि रतेरजस्रमश्रुस्रुतेर्योऽजनि सूत्रधारः ॥ २ ॥,tanotu bhūtiṃ dahatādaghāni sa śūlino locanapāvako vaḥ | dhūmānabhijño'pi raterajasramaśrusruteryo'jani sūtradhāraḥ || 2 ||,लोकोत्तरप्रभावत्वात्समिन्निरपेक्षमेव ज्वलन्नत एव धूमस्यानभिज्ञोऽपि यो रतेः कामवध्वा बाष्पप्रवाहस्य सूचनात्सूत्रधारः समपादि स हरस्य नेत्राग्निर्भूतिं विस्तारयतु । पापानि दहतात् । भूतिरैश्वर्यं भस्म...,"May the fire of Shiva’s (third) eye grant you prosperity/sacred ash and burn down your sins – a fire that, in spite of being smokeless, stage-managed Rati’s incessant shedding of tears.",Shiva's smokeless ocular fire blazes forth in the absence of a feeding fuel due to its divine power. The figure of speech employed in this verse is ‘vibhāvanā’ since the effect - the production of...
3,3.0,1.0,3,भालस्थलीरङ्गतले मृडस्य हुताशनस्ताण्डवकृत्स वोऽव्यात् । यस्मिन्रतिप्राणसमः शरीरमुन्मालकायेव निजं मुमोच ॥ ३ ॥,bhālasthalīraṅgatale mṛḍasya hutāśanastāṇḍavakṛtsa vo'vyāt | yasminratiprāṇasamaḥ śarīramunmālakāyeva nijaṃ mumoca || 3 ||,रतेः प्राणसमः प्रियः कामो नृत्तप्रीतः सन्नुन्मालकाय पारितोषकदानायेव शरीरं यस्मिंस्तत्त्याज स पावको हरस्य ललाटभूमिरेव रङ्गतलं तत्र ताण्डवमिव ताण्डवं कर्ता युष्मान्प्रीणयतु । रतिप्राणसम इति पर्यायवक...,"May the Fire, dancing a robust dance on the stage that is Shiva’s forehead, save us. Kama, Rati’s sweetheart, offered his very body to it (i.e., the dancer that is fire) as a gift, so it seemed.","Unmālaka, a rare word, is explained by JR as ‘pāritoṣakapradāna’ (‘offering a gift’). Mankha uses the word ‘ratiprāṇasama’ (‘as dear as life to Rati’) for Kama to highlight his amorous nature and,..."
4,4.0,1.0,4,तन्नेत्रमासूत्रितवह्निसख्यमाख्यातु वः शर्म शिवस्य यस्मिन् । ज्वालावलीपल्लवकेलितल्पे विनैव रत्या मदनो निदद्रौ ॥ ४ ॥,tannetramāsūtritavahnisakhyamākhyātu vaḥ śarma śivasya yasmin | jvālāvalīpallavakelitalpe vinaiva ratyā madano nidadrau || 4 ||,यस्मिन्नेत्रे ज्वालावल्येव वर्णसन्निवेशाभ्यां पल्लवलीलाशयनं तत्र रतिरहितः कामो निदद्रौ दग्धः। तदासूत्रितमुट्टङ्कितं वह्निना सख्यं येन तच्छिवसम्बन्धि नेत्रं वः सुखं प्रथयताम् । मदन इति पर्यायवक्रतय...,"Let Shiva’s eye – one that has forged a friendship with fire – proclaim happiness to you. In the pleasure bed of fresh leaves formed by its (i.e., the eye’s) billowing flames, Kama fell into sleep...","The flames resemble fresh leaves both in shape and colour. The word 'madana' is indicative of Kama's amorous nature, justifying thereby the use of 'eva' ('ratyā vinā eva', 'even without Rati'), i...."
...,...,...,...,...,...,...,...,...
1694,1694.0,25.0,149,सारस्वतस्य महसः प्रथमा शिखेव सा मङ्खकस्य ववृते धुरि सूक्तिरेका । दग्धुं मनः खलजनस्य कपोलराग-भङ्ग्या व्यधत्त वचनेष्विव या स्व​​​​मोजः ॥ १४९ ॥,sārasvatasya mahasaḥ prathamā śikheva sā maṅkhakasya vavṛte dhuri sūktirekā | dagdhuṃ manaḥ khalajanasya kapolarāga-bhaṅgyā vyadhatta vacaneṣviva yā svamojaḥ || 149 ||,सरस्वतीतेजोभिनवा ज्वालेव सा मङ्खकस्य सूक्तिः । या दुर्जनानां चित्तं दग्धुमिव मत्सरोत्थितगण्डारुणत्वव्याजान्निजं 

In [83]:
acc=[]
for elem in sc2.fillna(0).itertuples() :
    lines = []
    if elem.sloka_number ==0  :
        if len(elem.sloka) < 20 :
            lines += [ f'<sarga>\n{elem.sloka}\n</sarga>\n']
        else :
            lines += [ f'<pushpika>\n{elem.sloka}\n</pushpika>\n']
    else :
        try :
            sloka = re.sub(r'।\s*', '।\n', elem.sloka)
            lines += [f'''<sloka>\n{sloka}\n</sloka>\n''']
        except :
            # print(f"Sloka Error {elem.sloka}")
            pass

        try:
            transliterate = re.sub(r'\s\|\s', r' |\n', elem.transliterated)
            lines += [f'''<transliterate>\n{transliterate}\n</transliterate>\n''']
        except :
            # print(f"Transliterate Error {elem.transliterated}")
            pass
        try:
            # vyakhya = re.sub(r'।\s*', '।\n', elem.vyakhya)
            lines += [f'''<vyakhya>\n{vyakhya}\n</vyakhya>\n''']
        except :
            # print(f"Vyakhya Error {elem.vyakhya}")
            pass

        try:
            # translation = re.sub(r'।\s*', '।\n', elem.translation)
            lines += [f'''<translation>\n{translation}\n</translation>\n''']
        except :
            # print(f"Translation Error {elem.translation}")
            pass

        try:
            # fn_text = re.sub(r'।\s*', '।\n', elem.fn_text)
            lines += [f'''<fn-text>\n{fn_text}\n</fn-text>\n''']
        except :
            # print(f"Fn_text Error {elem.fn_text}")
            pass

        lines += ['<hr>\n']

    acc += lines
    

# read sc.css file to a string
with open('./sc.css', 'r') as f:
    css = f.read()


with open('./sc.html', 'w') as f:
    f.write("\n".join([css]+ ['<body>'] + acc + ['</body></html>']))
